In [28]:
from selenium import webdriver
from selenium.webdriver import FirefoxOptions
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import os
import time
import pandas as pd

URL = "https://www.rec-registry.gov.au/rec-registry/app/public/stc-register"
base_dir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))+'/'

In [29]:
# Setup the browser
opts = FirefoxOptions()
opts.add_argument("--headless")
driver = webdriver.Firefox(options=opts)

# Load the webpage
driver.get(URL)

# Find the search field
accreditation_code_field = driver.find_element(By.ID, 'accreditationCode')  # adjust the locator if it's not accurate

# Paste in the accreditation code and press enter
accreditation_code_field.send_keys('PVD4069315')
accreditation_code_field.send_keys(Keys.ENTER)

# Wait for the page to load
time.sleep(6) 

# Extract table rows and headers of search results table
s_headers = [header.text for header in driver.find_elements(By.XPATH, '//*[@id="search-ranges-results"]/thead/tr/th')]
s_rows = driver.find_elements(By.XPATH, '//*[@id="search-ranges-results"]/tbody/tr')

search_results_data = []
transfer_history_data = []

for s_row in s_rows:
    # Get the search resaults table data
    s_cols = s_row.find_elements(By.TAG_NAME, 'td') 
    s_cols_data = [ele.text for ele in s_cols]
    search_results_data.append(s_cols_data)

    # Click the link in the initial table to open the transfer history popup
    link = s_cols[7].find_element(By.TAG_NAME, 'a')
    link.click()

    # Get the transfer history table data
    t_headers = [header.text for header in driver.find_elements(By.XPATH, '//*[@id="transfer-history-results"]/thead/tr/th')]
    t_rows = driver.find_elements(By.XPATH, '//*[@id="transfer-history-results"]/tbody/tr')

    for t_row in t_rows:
         t_cols = t_row.find_elements(By.TAG_NAME, 'td') 
         t_cols_data = [ele.text for ele in t_cols]
         transfer_history_data.append(t_cols_data)
  
    # Close the transfer history popup
    html = driver.find_elements(By.XPATH, '//*[@id="certificate-details-dialog"]')[0]
    html.send_keys(Keys.ESCAPE)
   
# Close the browser
driver.close()


In [30]:
search_results_df = pd.DataFrame(search_results_data, columns=s_headers)
transfer_history_data_df = pd.DataFrame(transfer_history_data, columns=t_headers)

print(search_results_df)
print(transfer_history_data_df)

,Current owner,Accreditation code,Fuel source,Generation year,Creation year,Generation state,Status,Certificate serial number,Certificate quantity
0,Red Energy Pty. Limited,PVD4069315,S.G.U. - solar (deemed),2021,2021,WA,Invalid due to surrender,1-70,70
1,Red Energy Pty. Limited,PVD4069315,S.G.U. - solar (deemed),2021,2021,WA,Invalid due to surrender,71-91,21
